In [ ]:
import numpy as np
import xarray as xr

# Define input file paths
nc_file = "/Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200411.V8R_CH4_261.nc"
altout_file = "/Users/tobias/Downloads/Mipas/altout.nc"

# Run ncwa command
!ncwa -C -v altitude -a time {nc_file} {altout_file}

# Run ncrename command
!ncrename -h -v altitude,height {nc_file} renametest.nc

# Open the netCDF files as xarray datasets
targett = xr.open_dataset("renametest.nc")
altt = xr.open_dataset("altout.nc")

# Merge the two xarray datasets
ds = targett.merge(altt)

# Resample time to monthly intervals
monthly_time = ds.time.resample(time="1M").mean().values

# Define the bin edges for latitude and longitude
lat_bins = np.linspace(-90, 90, 19)
lon_bins = np.linspace(-180, 180, 37)

# Bin the latitude and longitude values
lat_indices = np.digitize(ds.latitude.values, lat_bins)
lon_indices = np.digitize(ds.longitude.values, lon_bins)

# Initialize an empty array to store the binned CH4 values
n_altitudes = ds.height.shape[0]
n_lats = len(lat_bins) - 1
n_lons = len(lon_bins) - 1
n_time = 1
ch4_binned = np.empty((n_altitudes, n_lats, n_lons, n_time))
ch4_binned[:] = np.nan

# Loop over altitudes and fill in the binned CH4 values
for alt_idx in range(n_altitudes):
    ch4_alt = ds.target.values[alt_idx]
    for lat_idx in range(n_lats):
        for lon_idx in range(n_lons):
            mask = (lat_indices == lat_idx+1) & (lon_indices == lon_idx+1)
            if np.any(mask):
                ch4_binned[alt_idx, lat_idx, lon_idx, 0] = np.nanmean(ch4_alt[mask])
            else:
                ch4_binned[alt_idx, lat_idx, lon_idx, 0] = np.nan

# Create a new xarray dataset with the binned CH4 values
ds_binned = xr.Dataset(
    {"ch4": (["altitude", "latitude", "longitude", "time"], ch4_binned)},
    coords={
        "altitude": ds.height.values,
        "latitude": lat_bins[:-1] + np.diff(lat_bins)/2,
        "longitude": lon_bins[:-1] + np.diff(lon_bins)/2,
        "time": monthly_time
    },
    attrs=ds.attrs,
)


ncwa: /Users/tobias/Downloads/Mipas/altout.nc exists---`e'xit, `o'verwrite (i.e., clobber existing file), or `a'ppend (i.e., replace duplicate variables in, and add metadata and new variables to, existing file) (e/o/a)? 